In [11]:
import numpy as np

def simplex_proj(z):
    """
    Projection sur le probability simplex
    http://arxiv.org/pdf/1309.1541.pdf
    :return:
    """
    # for reshaping from matrix type
    y = np.array(z).reshape(len(z)) 
    D, = y.shape
    x = np.array(sorted(y, reverse=True))
    u = [x[j] + 1. / (j + 1) * (1 - sum([x[i] for i in range(j + 1)])) for j in range(D)]
    l = []
    for idx, val in enumerate(u):
        if val > 0 :
            l.append(idx)
    if l == []:
        l.append(0)
    rho = max(l)
    lambd = 1. / (rho + 1) * (1 - sum([x[i] for i in range(rho + 1)]))
    return np.array([max(yi + lambd, 0) for yi in y])

In [12]:
from scipy.stats import multivariate_normal


In [395]:
import os, sys
algo_root = '..'
sys.path.insert(0, algo_root)

In [14]:
from tools.gm_tools import gm_params_generator, gaussian_mixture_sample

In [15]:
def gradient_different_lambdas(X, means, covars, pi, lambd, EPSILON=1e-8):
    """
    Evaluate the gradient of -sum_i^n( log( sum_j^K (pi_j * phi(mu_j, sigma_j)(X_i) ))) + sum_l^K (lambda_l*pi_l)
    """
    densities = np.array([multivariate_normal.pdf(X, means[i], covars[i]) for i in range(len(pi))]).T
    #We reshape for the division and add EPSILON to avoid zero division
    #we add the lambda penality (SLOPE like)
    return -(densities/(((densities*pi).sum(axis=1)).reshape(X.shape[0],1) + EPSILON)).sum(axis=0) + lambd

In [16]:
def pi_differentLambdas_estim_fista(X, means, covars, pi, L, lambd):
    """
    We use FISTA to accelerate the convergence of the algorithm
    we project the next step of the gradient descent on the probability simplex
    """
    t_previous = 1
    pi_previous = np.copy(pi)
    xi = np.copy(pi_previous)
    # the number of iterations is given on FISTA paper, 
    # we took ||pi_hat-pi_star||**2 = len(pi)**2
    fista_iter = int(np.sqrt(2*len(pi)**2 * L) // 1)
    for _ in range(min(500, fista_iter)):
        pi_next = simplex_proj(xi - 1./(np.sqrt(X.shape[0])*L)*gradient_different_lambdas(X, means, covars, xi, lambd))
        t_next = (1. + np.sqrt(1 + 4 * t_previous**2)) / 2
        xi = pi_next + (t_previous - 1) / t_next * (pi_next - pi_previous)
        pi_previous = np.copy(pi_next)
    return pi_next

In [17]:
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.base import BaseEstimator
from sklearn.mixture import GMM
from sklearn.utils import check_array
from tools.matrix_tools import check_zero_matrix

In [18]:
def algo_different_lambdas_penalities_1(X, max_clusters, n_iter, L, alpha=0.01):
    """
    we inject in the gradient the penality, and project the estimate in the
    probability simplex
    """
    lambd = lambda_list_BH(max_clusters, alpha)
    # initialization of the algorithm
    g = GMM(n_components=max_clusters, covariance_type= "full")
    g.fit(X)
    means_estim, covars_estim, pi_estim = g.means_, g.covars_, g.weights_
    N = len(X)
    K = len(pi_estim)
    print "Init EM pi: ",pi_estim
    for it in range(n_iter):
        # We estimate pi according to the penalities lambdas given
        pi_estim = pi_differentLambdas_estim_fista(X, means_estim, covars_estim, pi_estim, L, lambd)
        # we remove the clusters with probability = 0
        non_zero_elements = np.nonzero(pi_estim)[0]
        K = len(non_zero_elements)
        pi_estim = np.array([pi_estim[i] for i in non_zero_elements])
        means_estim = np.array([means_estim[i] for i in non_zero_elements]) 
        covars_estim = np.array([covars_estim[i] for i in non_zero_elements])
        lambd = np.array([lambd[i] for i in non_zero_elements])
        # we estimate the conditional probability P(z=j/X[i])
        tau = tau_estim(X, means_estim, covars_estim, pi_estim)
        # Means
        means_estim = np.array([(tau[:, k]*X.T).sum(axis=1)*1/(N*pi_estim[k]) for k in range(K)])
        # covars 
        covars_temp = np.array(
                [covar_estim(X, means_estim[k], tau[:, k], pi_estim[k]) for k in range(K)])
        non_empty_covar_idx = check_zero_matrix(covars_temp)
        pi_estim = [pi_estim[j] for j in non_empty_covar_idx]
        means_estim = [means_estim[j] for j in non_empty_covar_idx]
        covars_estim = [covars_estim[j] for j in non_empty_covar_idx]
        lambd = [lambd[j] for j in non_empty_covar_idx]
        K = len(pi_estim)
        if it%10 == 0 :
            print "iteration ",it, "pi: ", pi_estim
    return pi_estim, means_estim, covar_estim, tau_estim

In [19]:
def tau_estim(X, centers, covars, pi):
    try:
        densities = np.array([multivariate_normal.pdf(X, centers[k], covars[k], allow_singular=True) for k in range(len(pi))]).T * pi
        return (densities.T/(densities.sum(axis=1))).T
    except np.linalg.LinAlgError as e:
        print "Error on density computation for tau", e

In [20]:
def covar_estim(X, mean, tau, pi):
    """
    emp covariance of EM
    :param mean: mean for one cluster
    :param pi: pi for this cluster
    :param N: lenth of X
    :param tau: vector of proba for each X[i] in the cluster, given by tau[:,k]
    :return: emp covariance matrix of this cluster
    """
    N = len(X)
    Z = np.sqrt(tau).reshape(N, 1) * (X - mean)
    return 1 / (pi * N) * Z.T.dot(Z)

In [21]:

def check_zero_matrix(mat_list):
    """
    Return the list of matrices ids which are non empty
    :param mat_list: List of matrices, usually covariance matrices
    :return: list of ids of non empty matrices
    """
    non_zero_list = []
    for i in range(len(mat_list)):
        if np.count_nonzero(mat_list[i]) is not 0:
            non_zero_list.append(i)
    return non_zero_list

In [22]:
#def main():
#    pi, means, covars = gm_params_generator(3,3)
#    X,_ = gaussian_mixture_sample(pi, means, covars, 1e5)
#    pi_e = algo_different_lambdas_penalities_1(X,max_clusters=5,n_iter=500, L=1e5)
#    return pi_e, pi
##pi_e, pi = main()
#print "real pi: ", pi
#print "estimated pi: ", pi_e

In [23]:
def lambda_list_BH(K, alpha=1):
    #Cf pierre bellec, Candes (Mimimax SLOPE), lambda_BH, we add a normalization
    return alpha*np.array([np.sqrt(2./K*np.log(1.*K/(j+1))) for j in range(K)])

# Alg. optim sur le cone + proj simplex

In [24]:
def simple_gradient(X, means, covars, pi, EPSILON=1e-8):
    """
    Evaluate the gradient of -1/n*sum_i^n( log( sum_j^K (pi_j * phi(mu_j, sigma_j)(X_i) ))) 
    """
    densities = np.array([multivariate_normal.pdf(X, means[i], covars[i]) for i in range(len(pi))]).T
    #We reshape for the division and add EPSILON to avoid zero division
    #we add the lambda penality (SLOPE like)
    return -1./X.shape[0]*(densities/(((densities*pi).sum(axis=1)).reshape(X.shape[0],1) + EPSILON)).sum(axis=0)

In [25]:
from cvxpy import *

def ordered_optim_proj(y, lambd):
    """
    We solve the optimization problem:
    1/2*||b-x||**2 + sum_j(lambda_i*x_j) with csts: x_1>=x_2>=...>=x_k>0 and sum(x_i) = 1
    """
    # Construct the problem.
    n = y.shape[0]
    x = Variable(n)    
    objective = Minimize(1./n*sum_squares(x - y) + sum_entries(np.diag(lambd)*x))
    #We reformulate the constrains as: x_i - x_j >= 0 i,j in [k-1] and x_k > 0
    constraints = [(x[:n-1]-x[1:])>=0, x[-1]>0, sum_entries(x)==1]
    prob = Problem(objective, constraints)
    # The optimal objective is returned by prob.solve().
    result = prob.solve(solver=CVXOPT)
    #We project on the probability simplex
    # The optimal value for x is stored in x.value.
    return np.array(x.value).reshape(len(x.value))


In [26]:
def pi_differentLambdas_estim_fista_conic(X, means, covars, pi, L, lambd):
    """
    We use FISTA to accelerate the convergence of the algorithm
    we project the next step of the gradient descent on the probability simplex
    """
    t_previous = 1
    pi_previous = np.copy(pi)
    xi = np.copy(pi_previous)
    # the number of iterations is given on FISTA paper, 
    # we took ||pi_hat-pi_star||**2 = len(pi)**2
    fista_iter = int(np.sqrt(2*len(pi)**2 * L) // 1)
    for _ in range(min(500, fista_iter)):
        pi_next = ordered_optim_proj(xi - 1./(np.sqrt(X.shape[0])*L)*simple_gradient(X, means, covars, xi), lambd)
        t_next = (1. + np.sqrt(1 + 4 * t_previous**2)) / 2
        xi = pi_next + (t_previous - 1) / t_next * (pi_next - pi_previous)
        pi_previous = np.copy(pi_next)
    return pi_next

In [27]:
def algo_different_lambdas_penalities_conic(X, max_clusters, n_iter, L, alpha=0.001):
    """
    we inject in the gradient the penality, and project the estimate in the
    probability simplex
    """
    lambd = lambda_list_BH(max_clusters, alpha)
    # initialization of the algorithm
    g = GMM(n_components=max_clusters, covariance_type= "full")
    g.fit(X)
    # we order for slope
    print 
    pi_estim, means_estim, covars_estim = map(list,zip(*(sorted(zip(g.weights_, g.means_, g.covars_))[::-1])))
    print pi_estim
    print "Init EM pi: ",pi_estim
    N = len(X)
    K = len(pi_estim)
    for it in range(n_iter):
        # We estimate pi according to the penalities lambdas given
        pi_estim = pi_differentLambdas_estim_fista_conic(X, means_estim, covars_estim, pi_estim, L, lambd)
        # we remove the clusters with probability = 0
        non_zero_elements = np.nonzero(pi_estim)[0]
        K = len(non_zero_elements)
        pi_estim = np.array([pi_estim[i] for i in non_zero_elements])
        means_estim = np.array([means_estim[i] for i in non_zero_elements]) 
        covars_estim = np.array([covars_estim[i] for i in non_zero_elements])
        lambd = np.array([lambd[i] for i in non_zero_elements])
        # we estimate the conditional probability P(z=j/X[i])
        tau = tau_estim(X, means_estim, covars_estim, pi_estim)
        # Means
        means_estim = np.array([(tau[:, k]*X.T).sum(axis=1)*1/(N*pi_estim[k]) for k in range(K)])
        # covars 
        covars_temp = np.array(
                [covar_estim(X, means_estim[k], tau[:, k], pi_estim[k]) for k in range(K)])
        non_empty_covar_idx = check_zero_matrix(covars_temp)
        pi_estim = [pi_estim[j] for j in non_empty_covar_idx]
        means_estim = [means_estim[j] for j in non_empty_covar_idx]
        covars_estim = [covars_estim[j] for j in non_empty_covar_idx]
        lambd = [lambd[j] for j in non_empty_covar_idx]
        K = len(pi_estim)
        if it%10 == 0 :
            print "iteration ",it, "pi: ", pi_estim
    return pi_estim, means_estim, covars_estim, tau_estim

In [28]:
def view2Ddata(X):
    from plotly.offline import plot
    import plotly.graph_objs as go
    
    # Create random data with numpy
    import numpy as np
    
    N = 1000
    random_x = np.random.randn(N)
    random_y = np.random.randn(N)
    
    # Create a trace
    trace = go.Scatter(
        x=X[:,0],
        y=X[:,1],
        mode = 'markers'
    )
    
    data = [trace]
    
    # Plot and embed in ipython notebook!
    plot(data, filename='Plot')

In [29]:
from sklearn.datasets import make_sparse_spd_matrix

def gm_params_generator(d, k, sparse_proba=None):
    """
    We generate centers in [-0.5, 0.5] and verify that they are separated enough
    """
    #  we scatter the unit square on k squares, the min distance is given by c/sqrt(k)
    min_center_dist = 0.1/np.sqrt(k)
    centers = [np.random.rand(1, d)[0]-0.5]
    for i in range(k-1):
        center = np.random.rand(1, d)[0]-0.5
        distances = np.linalg.norm(
            np.array(centers) - np.array(center),
            axis=1)
        while len(distances[distances < min_center_dist]) > 0:
            center = np.random.rand(1, d)[0]-0.5
            distances = np.linalg.norm(
                np.array(centers) - np.array(center),
                axis=1)
        centers.append(center)
    # if sparse_proba is set :
    #    generate covariance matrix with the possibility to set the sparsity on the precision matrix, 
    # we multiply by 1/k^2 to avoid overlapping
    if sparse_proba==None:
        A = [random.rand(d,d) for _ in range(k)]
        cov = [1e-2/(k**2)*(np.diag(np.ones(d))+np.dot(a,a.transpose())) for a in A]
    else:
        cov = np.array([np.linalg.inv(make_sparse_spd_matrix(d, alpha=sparse_proba)) for _ in range(k)])
    p = np.random.randint(1000, size=(1, k))[0]
    weights = 1.0*p/p.sum()
    return weights, centers, cov

In [30]:
pi, means, covars = gm_params_generator(2,3)
X,_ = gaussian_mixture_sample(pi, means, covars, 1e4)
view2Ddata(X)


# simulations

In [31]:
# methode SLOPE
# avec pi_i ordonnés
# avec 1/2*||b-x||**2 + sum_j(lambda_i*x_j) with csts: x_1>=x_2>=...>=x_k>0 and sum(x_i) = 1
#utilisation de CVXPY
pi_e, means_e, covars_e, _  = algo_different_lambdas_penalities_conic(X,max_clusters=5,n_iter=100, L=1e5, alpha=0.0001 )
print "real pi: ", pi
print "estimated pi: ", pi_e
print "real means", means
print "estimated means", means_e


[0.30169131291594298, 0.27996295018082018, 0.22744297633833038, 0.13650407707339524, 0.05439868349151232]
Init EM pi:  [0.30169131291594298, 0.27996295018082018, 0.22744297633833038, 0.13650407707339524, 0.05439868349151232]
iteration  0 pi:  [0.26467211524795647, 0.26417334677167775, 0.2368065377087514, 0.13320742999211746, 0.10114057027949686]


KeyboardInterrupt: 

In [ ]:
# ancienne methode avec les meme lambda_i lambda_i = alpha*sqrt(2*log(max_clusters/i))
# sans les pi_i ordonnés
pi_e_2, means_e_2, covars_e_2, _ = algo_different_lambdas_penalities_1(X,max_clusters=5,n_iter=100, L=1e4, alpha=1)
print "real pi: ", pi
print "estimated pi: ", pi_e_2
print "real means", means
print "estimated means", means_e_2

In [32]:
#La projection nous ramene sur le "l'angle" du simplex à la valeur 1/K
#En utilisant les lambda_BH_i donnés dans "SLOPE is adaptive to unknown sparsity and asymptotically minimax" (SU, Candes 2015) 
#conseillé par Pierre B.
#lambda_i = alpha*sqrt(2*log(max_clusters/i))
a = np.array([ 0.2933247 ,  0.09657283,  0.24179129 , 0.10878973 , 0.25952145])
ordered_optim_proj(a,lambda_list_BH(len(a)))

array([ 0.20000002,  0.19999999,  0.19999999,  0.2       ,  0.19999999])

In [33]:
#en donnant un alpha faible
a = np.array([ 0.2933247 ,  0.09657283,  0.24179129 , 0.10878973 , 0.25952145])
ordered_optim_proj(a,lambda_list_BH(len(a),alpha=0.01))

array([ 0.28405798,  0.17898558,  0.17898551,  0.1789855 ,  0.17898543])

In [34]:
#En utilisant les lambda_BH_i donnés dans "SLOPE is adaptive to unknown sparsity and asymptotically minimax" (SU, Candes 2015) 
#conseillé par Pierre B.
#lambda_i = alpha*sqrt(2*log(max_clusters/i))

print lambda_list_BH(5)
print lambda_list_BH(5, alpha=0.01)

[ 0.80235601  0.60540589  0.45202904  0.2987598   0.        ]
[ 0.00802356  0.00605406  0.00452029  0.0029876   0.        ]


In [35]:
#test en dim 2
print ordered_optim_proj(np.array([0.5,1]),np.array([1,1])) #en dehors de la projection sur la "face" du simplex, on arrive a l'angle
print ordered_optim_proj(np.array([0,1]),np.array([1,1])) # pareil
print ordered_optim_proj(np.array([1,0.5]),np.array([1,1]))
print ordered_optim_proj(np.array([0.5,1]),np.array([10,1]))
# ci dessus le comportement est normal
print ordered_optim_proj(np.array([0.5, 1, 0.5]),lambda_list_BH(3, alpha=1))
print ordered_optim_proj(np.array([0.5,1, 0.5]),lambda_list_BH(3, alpha=0.01))
print ordered_optim_proj(np.array([0.5,1, 0.5]),lambda_list_BH(3, alpha=0.001))

[ 0.50000006  0.49999994]
[ 0.50000008  0.49999992]
[ 0.75005225  0.24994775]
[ 0.5  0.5]
[ 0.33333334  0.33333333  0.33333333]
[ 0.41322082  0.41322076  0.17355842]
[ 0.41630399  0.41630392  0.16739208]


In [36]:
"""
Ci dessous on decompose la minimisation de :

1/2*||b-x||**2 + sum_j(lambda_i*x_j) avec les contraintes: x_1>=x_2>=...>=x_k>0 and sum(x_i) = 1

On va d'abord faire la minimisation avec les contraintes : x_1>=x_2>=...>=x_k>0
Puis faire une projection sur le simplex prob.
"""

"\nCi dessous on decompose la minimisation de :\n\n1/2*||b-x||**2 + sum_j(lambda_i*x_j) avec les contraintes: x_1>=x_2>=...>=x_k>0 and sum(x_i) = 1\n\nOn va d'abord faire la minimisation avec les contraintes : x_1>=x_2>=...>=x_k>0\nPuis faire une projection sur le simplex prob.\n"

In [37]:
def minimization(y, lambd):
    """
    We solve the optimization problem:
    1/2*||b-x||**2 + sum_j(lambda_i*x_j) with csts: x_1>=x_2>=...>=x_k>0 and sum(x_i) = 1
    """
    # Construct the problem.
    n = len(y)
    x = Variable(n)    
    objective = Minimize(1./n*sum_squares(x - y) + sum_entries(np.diag(lambd)*x))
    #We reformulate the constrains as: x_i - x_j >= 0 i,j in [k-1] and x_k > 0
    constraints = [x[-1]>0] + [x[i]>=0 for i in range(n-1)] 
    prob = Problem(objective, constraints)
    # The optimal objective is returned by prob.solve().
    result = prob.solve(solver=CVXOPT)
    #We project on the probability simplex
    # The optimal value for x is stored in x.value.
    return x.value

y = np.array([ 0.2933247 ,  0.19657283,  0.24179129 , 0.10878973 , 0.25952145])
y_temp = minimization(y, lambda_list_BH(len(y), alpha=0.01))
print y_temp

[[ 0.27326591]
 [ 0.1814368 ]
 [ 0.23049041]
 [ 0.1013161 ]
 [ 0.2595216 ]]


In [38]:
#On remarque
#On projette:
simplex_proj(y_temp)

array([ 0.26405975,  0.17223063,  0.22128424,  0.09210994,  0.25031543])

# lasso with square root

In [300]:
def proj_circle(v):
    """
    we receive a vector [v1,v2,...,vp] and project it on the unit circle.
    """
    return v/np.linalg.norm(v)

In [301]:
def grad_sqrt_penalty(X, means, covars, alpha, lambd, EPSILON=1e-8):
    """
    Evaluate the gradient of -1/n*sum_i^n( log( sum_j^K (alpha_j^2 * phi(mu_j, sigma_j)(X_i) ))) + lambd*sum_j^K (alpha_j)
    """
    densities = np.array([multivariate_normal.pdf(X, means[i], covars[i]) for i in range(len(alpha))]).T
    #We reshape for the division and add EPSILON to avoid zero division
    #we add the lambda penality 
    return -1./X.shape[0]*(densities*2*alpha/(((densities*alpha**2).sum(axis=1)).reshape(X.shape[0],1) + EPSILON)).sum(axis=0) + lambd

In [302]:
def pi_sqrt_lasso_estim_fista(X, means, covars, pi, L, lambd):
    """
    lasso with square root of pi estimation
    We use FISTA to accelerate the convergence of the algorithm
    we project the next step (squared) of the gradient descent on the unit circle
    """
    t_previous = 1
    alpha_previous = np.copy(pi)
    xi = np.copy(alpha_previous)
    # the number of iterations is given on FISTA paper, 
    # we took ||pi_hat-pi_star||**2 = len(pi)**2
    fista_iter = int(np.sqrt(2*len(pi)**2 * L) // 1)
    for _ in range(max(100, fista_iter)):
        alpha_next = proj_circle(xi - 1./(np.sqrt(X.shape[0])*L)*grad_sqrt_penalty(X, means, covars, xi, lambd))
        t_next = (1. + np.sqrt(1 + 4 * t_previous**2)) / 2
        xi = alpha_next + (t_previous - 1) / t_next * (alpha_next - alpha_previous)
        alpha_previous = np.copy(alpha_next)
    #We return the squared vector to obtain a probability vector sum = 1
    return alpha_next**2

In [303]:
def algo__sqrt_lasso(X, max_clusters, n_iter, L, lambd=1):
    """
    we inject in the gradient the penality, and project the estimate in the
    probability simplex
    """
    # initialization of the algorithm
    g = GMM(n_components=max_clusters, covariance_type= "full")
    g.fit(X)
    # we order for slope
    #pi_estim, means_estim, covars_estim = map(list,zip(*(sorted(zip(g.weights_, g.means_, g.covars_))[::-1])))
    means_estim, covars_estim, pi_estim = g.means_, g.covars_, g.weights_
    print "Init EM pi: ",pi_estim
    N = len(X)
    K = len(pi_estim)
    for it in range(n_iter):
        # We estimate pi according to the penalities lambdas given
        pi_estim = pi_sqrt_lasso_estim_fista(X, means_estim, covars_estim, pi_estim, L, lambd)
        # we remove the clusters with probability = 0
        non_zero_elements = np.nonzero(pi_estim)[0]
        K = len(non_zero_elements)
        pi_estim = np.array([pi_estim[i] for i in non_zero_elements])
        means_estim = np.array([means_estim[i] for i in non_zero_elements]) 
        covars_estim = np.array([covars_estim[i] for i in non_zero_elements])
        # we estimate the conditional probability P(z=j/X[i])
        tau = tau_estim(X, means_estim, covars_estim, pi_estim)
        # Means
        means_estim = np.array([(tau[:, k]*X.T).sum(axis=1)*1/(N*pi_estim[k]) for k in range(K)])
        # covars 
        covars_temp = np.array(
                [covar_estim(X, means_estim[k], tau[:, k], pi_estim[k]) for k in range(K)])
        non_empty_covar_idx = check_zero_matrix(covars_temp)
        pi_estim = [pi_estim[j] for j in non_empty_covar_idx]
        means_estim = [means_estim[j] for j in non_empty_covar_idx]
        covars_estim = [covars_estim[j] for j in non_empty_covar_idx]
        K = len(pi_estim)
        if it%10 == 0 :
            print "iteration ",it, "pi: ", pi_estim
    return pi_estim, means_estim, covars_estim, tau_estim

In [306]:
# methode (square root) lasso 
# avec pi_i non ordonnés
# avec -1/n*sum_i^n( log( sum_j^K (alpha_j^2 * phi(mu_j, sigma_j)(X_i) ))) + lambd*sum_j^K (alpha_j)
print "real pi: ", pi
pi_e, means_e, covars_e, _  = algo__sqrt_lasso(X,max_clusters=7,n_iter=100, L=1, lambd=1 )
print "estimated pi: ", pi_e
print "real means", means
print "estimated means", means_e

real pi:  [ 0.4005168   0.25374677  0.34573643]
Init EM pi:  [ 0.19611196  0.19028569  0.15186744  0.18118628  0.05988816  0.19313256
  0.02752792]
iteration  0 pi:  [0.13364902793129213, 0.13843301205467257, 0.16409573000553776, 0.1406961647416419, 0.13424469820574425, 0.15705899289162592, 0.13182237416948539]
iteration  10 pi:  [0.093859060182531082, 0.35813637731986703, 0.13317731888787929, 0.12822109065347748, 0.28660615295624525]
iteration  20 pi:  [0.20491729427780861, 0.44751164416117389, 0.34757106156101769]
iteration  30 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760777323457126]
iteration  40 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760777323457126]
iteration  50 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760777323457126]
iteration  60 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760777323457126]
iteration  70 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760777323457126]
iteration  80 pi:  [0.20498526358504812, 0.4474069631803807, 0.34760

# square root lasso en faisant sur p-1

In [368]:
pi, means, covars = gm_params_generator(2,3)
means = [np.array([0,0]), np.array([1,1]), np.array([0,2])]
X,_ = gaussian_mixture_sample(pi, means, covars, 1e4)

In [369]:
view2Ddata(X)

In [310]:
def proj_unit_disk(v):
    """
    we receive a vector [v1,v2,...,vp] and project [v1,v2,...,vp-1] on the unit disk.
    """
    if np.linalg.norm(v)**2 <= 1:
        return v
    else:
        return v/np.linalg.norm(v)

In [311]:
def grad_sqrt_penalty(X, means, covars, alpha, lambd, EPSILON=1e-8):
    """
    alpha is of dim p-1
    density is of dim p-1
    Evaluate the gradient of 
    """
    dens_last_comp = multivariate_normal.pdf(X, means[len(alpha)], covars[len(alpha)]).reshape(X.shape[0],1)
    dens_witht_p_comp = np.array([multivariate_normal.pdf(X, means[i], covars[i]) for i in range(len(alpha))]).T - dens_last_comp 
    #We reshape for the division and add EPSILON to avoid zero division
    #we add the lambda penality 
    return lambd - 2./X.shape[0]*(alpha*dens_witht_p_comp/(EPSILON+dens_last_comp+((alpha**2)*dens_witht_p_comp).sum(axis=1).reshape(X.shape[0],1))).sum(axis=0)

In [354]:
def pi_sqrt_lasso_reduced_estim_fista(X, means, covars, pi, L, lambd):
    """
    lasso with square root of pi estimation
    We use FISTA to accelerate the convergence of the algorithm
    we project the next step (squared) of the gradient descent on the unit circle
    """
    t_previous = 1
    #we delete the last element and take the square root of the vector
    alpha_previous = np.copy(np.sqrt(pi[:-1]))
    xi = np.copy(alpha_previous)
    # the number of iterations is given on FISTA paper, 
    # we took ||pi_hat-pi_star||**2 = len(pi)**2
    fista_iter = int(np.sqrt(2*len(pi)**2 * L) // 1)
    for _ in range(500):
        grad_step = xi - 1./(np.sqrt(X.shape[0])*L)*grad_sqrt_penalty(X, means, covars, xi, lambd)
        alpha_next = proj_unit_disk(grad_step)
        t_next = (1. + np.sqrt(1 + 4 * t_previous**2)) / 2
        xi = alpha_next + (t_previous - 1) / t_next * (alpha_next - alpha_previous)
        alpha_previous = np.copy(alpha_next)
    #We return the squared vector to obtain a probability vector sum = 1
    return np.append(alpha_next**2, max(0,1-np.linalg.norm(alpha_next)**2))

In [355]:
def algo__sqrt_reduced_lasso(X, max_clusters, n_iter, L, lambd=1):
    """
    we inject in the gradient the penality, and project the estimate in the
    probability simplex
    """
    # initialization of the algorithm
    g = GMM(n_components=max_clusters, covariance_type= "full")
    g.fit(X)
    # we order for slope
    #pi_estim, means_estim, covars_estim = map(list,zip(*(sorted(zip(g.weights_, g.means_, g.covars_))[::-1])))
    means_estim, covars_estim, pi_estim = g.means_, g.covars_, g.weights_
    print "Init EM pi: ",pi_estim
    N = len(X)
    K = len(pi_estim)
    for it in range(n_iter):
        # We estimate pi according to the penalities lambdas given
        pi_estim = pi_sqrt_lasso_reduced_estim_fista(X, means_estim, covars_estim, pi_estim, L, lambd)
        # we remove the clusters with probability = 0
        non_zero_elements = np.nonzero(pi_estim)[0]
        K = len(non_zero_elements)
        pi_estim = np.array([pi_estim[i] for i in non_zero_elements])
        means_estim = np.array([means_estim[i] for i in non_zero_elements]) 
        covars_estim = np.array([covars_estim[i] for i in non_zero_elements])
        # we estimate the conditional probability P(z=j/X[i])
        tau = tau_estim(X, means_estim, covars_estim, pi_estim)
        # Means
        means_estim = np.array([(tau[:, k]*X.T).sum(axis=1)*1/(N*pi_estim[k]) for k in range(K)])
        # covars 
        covars_temp = np.array(
                [covar_estim(X, means_estim[k], tau[:, k], pi_estim[k]) for k in range(K)])
        non_empty_covar_idx = check_zero_matrix(covars_temp)
        pi_estim = [pi_estim[j] for j in non_empty_covar_idx]
        means_estim = [means_estim[j] for j in non_empty_covar_idx]
        covars_estim = [covars_estim[j] for j in non_empty_covar_idx]
        K = len(pi_estim)
        if it%10 == 0 :
            print "iteration ",it, "pi: ", pi_estim
    return pi_estim, means_estim, covars_estim, tau_estim

In [370]:
# methode (square root) lasso 
# avec pi_i non ordonnés
max_clusters = 5
lambd = np.sqrt(2*np.log(max_clusters)/X.shape[0])
print lambd
print "real pi: ", pi
pi_e, means_e, covars_e, _  = algo__sqrt_reduced_lasso(X,max_clusters=max_clusters, n_iter=100, L=1, lambd=lambd)
print "estimated pi: ", pi_e
print "real means", means
print "estimated means", means_e

0.0179412257799
real pi:  [ 0.32752613  0.56329849  0.10917538]
Init EM pi:  [ 0.13298734  0.25140046  0.1084      0.19421266  0.31299954]
iteration  0 pi:  [0.12079342944343699, 0.14153310053961654, 0.10642280622088783, 0.20213441501846238, 0.42911624877759624]
iteration  10 pi:  [0.0017190703487969773, 0.008225589847682839, 0.10621187654239497, 0.31556205807308929, 0.56828140518803605]
iteration  20 pi:  [0.00096353973676235806, 0.0063777873030011365, 0.10623370987653039, 0.31802377658496167, 0.56840118649874449]
iteration  30 pi:  [0.00012194930200124043, 0.0090047990496317631, 0.10625726831427418, 0.31608539515516365, 0.56853058817892921]
iteration  40 pi:  [9.4543129696924623e-06, 0.006641229693130097, 0.10626433704947924, 0.31851559478546843, 0.56856938415895253]
iteration  50 pi:  [0.00012296537012650093, 0.0082349217723062251, 0.10626178310586859, 0.31682495151267109, 0.56855537823902758]
iteration  60 pi:  [1.0801529443002688e-05, 0.0064205703884382924, 0.10626597661036494, 0.

In [118]:
np.array([ 0.27233861 , 0.46956135,  0.44069292 , 0.26629875])**2

array([ 0.07416832,  0.22048786,  0.19421025,  0.07091502])

In [288]:
def test_grad(x):
    xi = np.copy(x)
    L = 1
    lambd = 0
    for _ in range(100):
        xi = xi - 1./(np.sqrt(X.shape[0])*L)*grad_sqrt_penalty(X, means, covars, xi, lambd)
    return xi

In [289]:
test_grad([0.5,0.8])

array([ 0.63166424,  0.50596471])

In [290]:
x = np.linspace(0, 1.0, num=11)
y = np.linspace(0, 1.0, num=11)
e = np.array(zip(x,y))

In [333]:
for el in e:
    print test_grad(el)

[ 0.  0.]
[ 0.46502424  0.65454385]
[ 0.46535442  0.65431305]
[ 0.46580468  0.6539983 ]
[ 0.46628565  0.65366055]
[ 0.46669771  0.65336969]
[ 0.46696407  0.65318082]
[ 0.46654868  0.65347478]
[ 1.578448    1.89279789]
[ 1.61203306  1.9144976 ]
[ 1.65791197  1.94695758]


# Cross validation + gridsearch_cv

In [3]:
%load_ext autoreload
%autoreload 2
import os, sys
algo_root = '..'
sys.path.insert(0, algo_root)
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from tools.gm_tools import gm_params_generator, gaussian_mixture_sample, covar_estim, score, tau_estim
from tools.algorithms_benchmark import view2Ddata
from tools.gm_tools import score
from cluster.sq_root_lasso import sqrt_lasso_gmm
from sklearn.mixture import GMM

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [228]:
pi, means, covars = gm_params_generator(2,3)
X,_ = gaussian_mixture_sample(pi, means, covars, 1e5)
#view2Ddata(X)

In [229]:
X_train, X_validation, y_train, y_test = train_test_split(
    X, np.zeros(len(X)), test_size=0.2, random_state=0)

In [ ]:
#grid search on sq_root_lasso method
max_clusters = 7
lambd = np.sqrt(2*np.log(max_clusters)/X_train.shape[0])
param = {"lambda_param":[lambd, lambd+1e-1, lambd+1, lambd+10, lambd+100], "Lipshitz_c":[1, 10, 100, 1000, 10000], "max_clusters":[max_clusters]}
clf = GridSearchCV(estimator=sqrt_lasso_gmm(), param_grid=param, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
#we define a bic scoring method for the grid search
def bic_scorer(estimator, X, y=None):
    return (-2*score(X, estimator.weights_, estimator.means_, estimator.covars_ ) +
            estimator._n_parameters()*np.log(X.shape[0]))
params_GMM={"n_components":range(2,8), "covariance_type":["full"]}
clf_gmm = GridSearchCV(GMM(), param_grid=params_GMM, cv=5, n_jobs=-1, scoring=bic_scorer)
clf_gmm.fit(X_train)


In [ ]:
#we evaluate the loglikelihood of the fitted models on X_validation
print "real loglikelihood: ", 1/X_validation.shape[0]*score(X_validation, pi, means, covars)
print "###sq_root lasso method###"
print "sq_root lasso method loglikelihood:", 1/X_validation.shape[0]*score(X_validation, clf.best_estimator_.pi_, clf.best_estimator_.means_, clf.best_estimator_.covars_)
print "pi: ", clf_gmm.best_estimator_.weights_
print "means: ", clf_gmm.best_estimator_.means_
print "###EM###"
print "EM loglikelihood:", 1/X_validation.shape[0]*score(X_validation, clf_gmm.best_estimator_.weights_, clf_gmm.best_estimator_.means_, clf_gmm.best_estimator_.covars_)
print "pi: ", clf_gmm.best_estimator_.weights_
print "means: ", clf_gmm.best_estimator_.means_
